In [93]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import json
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import base64
# import errors

In [2]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        # Call the Gmail API
        service = build('gmail', 'v1', credentials=creds)
        results = service.users().labels().list(userId='me').execute()
        labels = results.get('labels', [])

        if not labels:
            print('No labels found.')
            return
        print('Labels:')
        for label in labels:
            print(label['name'])

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f'An error occurred: {error}')



In [3]:
! export redirect_uri=http://localhost:49737

In [5]:
# main()

# credentials file

In [24]:
json.load(open(SERVICE_ACCOUNT_FILE))

{'web': {'client_id': '356899316631-q9ra8imf3kusqksfbt17s09i0dsqmec0.apps.googleusercontent.com',
  'project_id': 'wagon-555-305810',
  'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
  'token_uri': 'https://oauth2.googleapis.com/token',
  'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
  'client_secret': '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'}}

In [49]:
SERVICE_ACCOUNT_FILE = json.load(open('service_acc.json'))
# SERVICE_ACCOUNT_FILE['client_email'] = 'moritz.georg.geiger@gmail.com'
# SERVICE_ACCOUNT_FILE['private_key'] = '124'
SCOPES = ['https://mail.google.com/']
# SERVICE_ACCOUNT_FILE

In [50]:
credentials = service_account.Credentials.from_service_account_info(
    SERVICE_ACCOUNT_FILE, 
    scopes=SCOPES
)

delegated_credentials = credentials.with_subject("moritz.georg.geiger@gmail.com")

service = build(
    'gmail', 
    'v1', 
    credentials=delegated_credentials, 
    cache_discovery=False
)

In [51]:
results = service.users().labels().list(userId='me').execute()
# labels = results.get('labels', [])
results

RefreshError: ('unauthorized_client: Client is unauthorized to retrieve access tokens using this method, or client not authorized for any of the scopes requested.', {'error': 'unauthorized_client', 'error_description': 'Client is unauthorized to retrieve access tokens using this method, or client not authorized for any of the scopes requested.'})

# email

In [96]:
def create_message(sender, to, subject, message_text):
    """Create a message for an email.

    Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

    Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    b64_bytes = base64.urlsafe_b64encode(message.as_bytes())
    b64_string = b64_bytes.decode()
    body = {'raw': b64_string}

    return body


In [97]:
def send_message(service, user_id, message):
    """Send an email message.

    Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.

    Returns:
    Sent Message.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                   .execute())
        print('Message Id: %s' % message['id'])
        return message
    except Exception as error:
        print('An error occurred: %s' % error)

In [98]:
sender = 'moritz.georg.geiger@gmail.com'
to = 'moritzgeiger@hotmail.de'
subject = 'test service acc'
message_text = 'hierhallohier'
user_id = 'moritz.georg.geiger@gmail.com'

message = create_message(sender, to, subject, message_text)
send_message(service, user_id, message)

An error occurred: ('unauthorized_client: Client is unauthorized to retrieve access tokens using this method, or client not authorized for any of the scopes requested.', {'error': 'unauthorized_client', 'error_description': 'Client is unauthorized to retrieve access tokens using this method, or client not authorized for any of the scopes requested.'})


In [99]:
message

{'raw': 'Q29udGVudC1UeXBlOiB0ZXh0L3BsYWluOyBjaGFyc2V0PSJ1cy1hc2NpaSIKTUlNRS1WZXJzaW9uOiAxLjAKQ29udGVudC1UcmFuc2Zlci1FbmNvZGluZzogN2JpdAp0bzogbW9yaXR6Z2VpZ2VyQGhvdG1haWwuZGUKZnJvbTogbW9yaXR6Lmdlb3JnLmdlaWdlckBnbWFpbC5jb20Kc3ViamVjdDogdGVzdCBzZXJ2aWNlIGFjYwoKaGllcmhhbGxvaGllcg=='}